In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from astropy.table import Table
from itertools import combinations
from astropy.coordinates import SkyCoord
from astropy import units as u


save = False

sns.set(style="whitegrid")

## Example

## INPUT: Gaia-style observables

You give `SkyCoord` the usual 6D astrometric quantities:

* `ra`, `dec`: right ascension and declination (angular position on the sky)
* `parallax`: gives distance → $d = \frac{1000}{\text{parallax in mas}}$
* `pmra`, `pmdec`: proper motions (tangential angular speed)
* `radial_velocity`: line-of-sight speed

---

## What SkyCoord Does Internally

### 1. **Position (x, y, z)**

It transforms your angular position + distance into Cartesian coordinates in the **ICRS (equatorial) frame**, centered on the Sun:

$$
\begin{aligned}
x &= d \cdot \cos(\text{dec}) \cdot \cos(\text{ra}) \\
y &= d \cdot \cos(\text{dec}) \cdot \sin(\text{ra}) \\
z &= d \cdot \sin(\text{dec})
\end{aligned}
$$

where `ra` and `dec` are in radians, and `d` is in parsecs.

### 2. **Velocity (vx, vy, vz)**

It uses the radial velocity and the proper motions to get the 3D space velocity:

* Proper motions are angular → converted into **km/s** using:

  $$
  v = 4.74047 \times \mu \times d
  $$
* It then projects these tangential and radial velocities into Cartesian components, aligned with the $x, y, z$ from above.

---

## Example:

If you do:

```python
c = SkyCoord(ra=..., dec=..., distance=..., pm_ra_cosdec=..., pm_dec=..., radial_velocity=...)
```

Then:

```python
c.cartesian.xyz  → gives (x, y, z) in pc  
c.velocity.d_xyz → gives (vx, vy, vz) in km/s
```

All in a consistent 3D Cartesian frame, with origin at the Sun and axes aligned with the ICRS system:

* $x$ toward the vernal equinox (RA=0)
* $z$ toward celestial north pole (Dec=+90°)
* $y$ completes the right-handed system

---


## Let's try with our data

In [26]:
#Gaia_df = pd.read_csv('DATA/My_Pleiades_filtered_Gaia_DR3_with_mass.csv')
binaries_candidates_df = pd.read_csv('DATA/My_Pleiades_binary_pairs.csv')

binaries_candidates_df.head(20)

,primary_source_id,secondary_source_id
0,66507469798631936,66507469798632320
1,68364544933829376,68364544935515392
2,64956123313498368,64956127609464320
3,66733552578791296,66733556873061120
4,65207709611941376,65207709613871744
5,66798496781121792,66798526845337344
6,65241313435901568,65241313437941504
7,65247704349267584,65248460263511552
8,65266494828710400,65266499126062080
9,65272817023559040,65272821318002560


In [27]:
# taking the binaries candidates from the Gaia DR3
#binaries_candidates_index1 = binaries_candidates_df['primary_source_id'].values
#binaries_candidates_index2 = binaries_candidates_df['secondary_source_id'].values

# Get the Gaia DR3 data for the binary candidates
binaries_1 = pd.read_csv('DATA/My_primary_members_binary_Pleiades_filtered.csv')
binaries_2 = pd.read_csv('DATA/My_secondary_members_binary_Pleiades_filtered.csv')

# orbits parameters
orbit_params = ['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity', 'Mass (M_sun)', 'ra_error', 'dec_error', 'pmra_error', 'pmdec_error', 'parallax_error', 'radial_velocity_error']
#orbit_params = ['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity', 'mass', 'ra_error', 'dec_error', 'pmra_error', 'pmdec_error', 'parallax_error', 'radial_velocity_error']

# Get the orbit parameters for the binary candidates
binaries_members_1 = binaries_1[orbit_params].copy()
binaries_members_2 = binaries_2[orbit_params].copy()

print(binaries_members_1.shape)
print(binaries_members_2.shape)

print('Check if the binaries candidates df contains Nan values:')
print(binaries_members_1.isnull().values.any())
print(binaries_members_2.isnull().values.any())
print(binaries_members_1.isna().values.any())
print(binaries_members_2.isna().values.any())

# Display the first few rows of the binaries members DataFrame

binaries_members_1.head()

(10, 14)
(10, 14)
Check if the binaries candidates df contains Nan values:
False
False
False
False


,source_id,ra,dec,pmra,pmdec,parallax,radial_velocity,Mass (M_sun),ra_error,dec_error,pmra_error,pmdec_error,parallax_error,radial_velocity_error
0,66507469798631936,57.491184,23.847947,21.026666,-48.132843,7.361154,4.707757,5.001,0.033703,0.018977,0.038665,0.025343,0.033577,0.533899
1,68364544933829376,55.515926,24.712536,21.721589,-45.292485,7.485806,12.204306,0.600,0.042979,0.035992,0.067268,0.057571,0.057112,4.892858
2,64956123313498368,56.453032,23.147851,19.029377,-45.463108,7.404870,5.299024,5.001,0.018650,0.011918,0.024697,0.014803,0.018652,0.219587
3,66733552578791296,56.655542,24.343339,22.380244,-45.516188,7.410337,6.865205,0.750,0.063617,0.040097,0.108746,0.062642,0.060509,3.227011
4,65207709611941376,56.851821,23.914478,20.049108,-44.132590,7.220987,-1.856786,2.600,0.030519,0.022987,0.041599,0.029272,0.034383,2.441681


## Monte Carlo sampling for the error

Since we have only one measurament we apply a MCM sampling to retrive the values of each orbit parameters and its error

| Parameter       | Range Chosen       | Why                                                                                 |
| --------------- | ------------------ | ----------------------------------------------------------------------------------- |
| `parallax`      | (0, 20) mas        | Pleiades parallax is \~7.4 mas. 20 mas is generous and avoids unphysical negatives. |
| `pmra`, `pmdec` | (-100, 100) mas/yr | Wide enough to include all reasonable proper motions for Gaia stars                 |
| `rv`            | (-100, 100) km/s   | Wide enough for almost all stellar RVs in the Galaxy                                |


In [30]:
import pandas as pd
import numpy as np
import emcee
from astropy.coordinates import SkyCoord
from astropy import units as u

np.random.seed(42)  # For reproducibility

# Define MCMC components
def log_prior(theta, obs, obs_err):
    ra, dec, plx, pmra, pmdec, rv = theta
    if 0 < plx < 20 and -100 < pmra < 100 and -100 < pmdec < 100 and -100 < rv < 100:
        return 0.0
    return -np.inf

def log_likelihood(theta, obs, obs_err):
    return -0.5 * np.sum(((theta - obs) / obs_err) ** 2)

def log_posterior(theta, obs, obs_err):
    lp = log_prior(theta, obs, obs_err)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, obs, obs_err)

def mcmc_sample_star(obs_vals, obs_errs, nwalkers=16, nsteps=300):
    ndim = len(obs_vals)
    p0 = obs_vals + 1e-4 * np.random.randn(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(obs_vals, obs_errs))
    sampler.run_mcmc(p0, nsteps, progress=False)
    samples = sampler.get_chain(discard=int(nsteps * 0.2), thin=10, flat=True)
    return samples

def summarize_samples(samples):
    param_names = ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity']
    summary = {}
    for i, name in enumerate(param_names):
        summary[f'{name}'] = np.mean(samples[:, i])
        summary[f'{name}_error'] = np.std(samples[:, i])
    return summary



def run_MCM_on_binaries(df, summary_list):
    for _, row in df.iterrows():
        obs = np.array([
            row['ra'], row['dec'], row['parallax'],
            row['pmra'], row['pmdec'], row['radial_velocity']
        ])
        obs_err = np.array([
            row['ra_error'], row['dec_error'], row['parallax_error'],
            row['pmra_error'], row['pmdec_error'], row['radial_velocity_error']
        ])
        samples = mcmc_sample_star(obs, obs_err, nwalkers=16, nsteps=1000)
        summary = summarize_samples(samples)
        summary_list.append(summary)
    return summary_list

# Create summary DataFrame both for binaries_members_1 and binaries_members_2
summary_list1 = []
summary_list1 = run_MCM_on_binaries(binaries_members_1, summary_list1)

summary_list2 = []
summary_list2 = run_MCM_on_binaries(binaries_members_2, summary_list2)

# Convert the summary list to a DataFrame
binaries_members_1_MCM = pd.DataFrame(summary_list1)
binaries_members_2_MCM = pd.DataFrame(summary_list2)

binaries_members_1_MCM.head(10)


,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,radial_velocity_error
0,57.491466,0.033305,23.848500,0.019122,7.363171,0.033386,21.021351,0.038177,-48.133672,0.025051,4.687575,0.501643
1,55.520199,0.043931,24.713419,0.036854,7.487410,0.055184,21.729241,0.066639,-45.293407,0.059256,12.071893,4.751178
2,56.452669,0.018192,23.147772,0.012233,7.406736,0.018276,19.030893,0.024709,-45.463276,0.014920,5.291427,0.212273
3,56.657712,0.061985,24.343744,0.041006,7.403738,0.060297,22.382387,0.111627,-45.516553,0.062315,6.960689,3.142841
4,56.850564,0.029533,23.915247,0.022421,7.221184,0.035806,20.048630,0.042990,-44.129799,0.029702,-1.524162,2.426270
5,56.477196,0.073291,24.551208,0.050857,7.303484,0.086374,20.331635,0.094621,-46.013909,0.060776,11.756057,0.571121
6,56.133926,0.037059,23.874283,0.028041,7.365840,0.038534,19.582492,0.046378,-45.428033,0.036195,16.332646,13.343356
7,56.053716,0.016187,24.031373,0.012069,7.594565,0.020427,21.404246,0.023067,-45.884287,0.015033,4.144967,0.523502
8,55.902495,0.069377,24.226303,0.057232,7.313981,0.079462,20.326572,0.086206,-46.512418,0.066202,-0.837315,7.962459
9,56.097853,0.034136,24.132501,0.024037,7.255867,0.039389,20.320970,0.040243,-47.789202,0.028737,10.145180,6.632449


In [31]:
## adding the corresponding source_id to the MCM results
binaries_members_1_MCM.insert(0, 'source_id', binaries_members_1['source_id'].values)
binaries_members_2_MCM.insert(0, 'source_id', binaries_members_2['source_id'].values)

#binaries_members_1_MCM.insert(1, 'Name', binaries_members_1['Name'].values)
#binaries_members_2_MCM.insert(1, 'Name', binaries_members_2['Name'].values)

binaries_members_1_MCM.insert(1, 'mass', binaries_members_1['Mass (M_sun)'].values)
binaries_members_2_MCM.insert(1, 'mass', binaries_members_2['Mass (M_sun)'].values)

binaries_members_2_MCM.head(10)

,source_id,mass,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,radial_velocity_error
0,66507469798632320,1.067,57.523066,0.244436,23.843829,0.128015,7.666924,0.229494,21.529037,0.290509,-43.576213,0.173142,1.119084,2.544196
1,68364544935515392,0.614,55.507484,0.162835,24.726694,0.154967,7.805989,0.195494,22.195642,0.225745,-46.619399,0.172733,5.352550,7.312751
2,64956127609464320,2.801,56.454640,0.040913,23.147303,0.026314,7.418069,0.042487,20.517538,0.052376,-45.699710,0.032644,7.721754,1.311818
3,66733556873061120,5.001,56.660814,0.076768,24.342126,0.053235,7.294581,0.077307,16.387006,0.156461,-44.770694,0.084155,4.186389,1.809551
4,65207709613871744,1.163,56.853242,0.046011,23.909874,0.036350,7.193719,0.054408,19.610298,0.058407,-46.330140,0.042595,5.340594,0.587084
5,66798526845337344,0.625,56.481867,0.022992,24.553933,0.014620,7.355720,0.023858,19.536592,0.030759,-45.548849,0.019328,4.438023,5.249725
6,65241313437941504,0.473,56.137939,0.035508,23.878460,0.025501,7.355039,0.034887,19.612429,0.048149,-46.532992,0.038160,10.174809,6.103585
7,65248460263511552,0.809,56.056028,0.090272,24.035772,0.091188,7.355366,0.133600,20.135138,0.128499,-45.155576,0.108745,4.713042,1.241585
8,65266499126062080,5.001,55.907287,0.101695,24.226607,0.074931,7.711384,0.111191,19.558893,0.125051,-42.105485,0.083811,-90.636398,5.837334
9,65272821318002560,1.395,56.101617,0.081201,24.138580,0.064729,7.375303,0.100563,28.074611,0.110852,-43.090645,0.077583,6.329138,0.407806


## ok Now we calculate the orbits parameters for each pair

Here we don't use the radial velocity since has a very large error

Orientation of the XYZ system (ICRS)
The Cartesian coordinate system in SkyCoord(..., frame='icrs') is defined as:

- X points toward RA = 0°, Dec = 0°

- Y points toward RA = 90°, Dec = 0°

- Z points toward the North Celestial Pole (Dec = +90°)

So, in ICRS:

- The XY plane is the celestial equator.

- Z points "up" in the sky toward Dec = +90°.


---

Let's unpack **why the node vector `n_vec = K × h_vec` uses `K = [0, 0, 1]`** (the unit vector along the z-axis) **even when the orbital plane is inclined**.

### 💡 Key Concept: Reference Frame vs Orbital Plane

#### ✅ You’re working in a **fixed inertial reference frame** (ICRS Cartesian, typically):

* **XY plane** = reference plane (celestial equator or ecliptic).
* **Z-axis** = normal to the reference plane (`K = [0, 0, 1]`).
* This frame **does not rotate** with the orbit.

---

### 🧭 Step-by-step logic

1. **Angular momentum vector `h_vec`**:

   * Points **perpendicular to the orbital plane**.
   * Its direction determines the orbit's **inclination `i`** with respect to the reference plane.

2. **Node vector `n_vec = K × h_vec`**:

   * Is a vector in the **reference plane (XY)**.
   * It points toward the **ascending node** — the point where the orbit **crosses the reference plane from below to above**.

3. `K` is fixed as the **reference z-axis**:

   * You're projecting the orbit's angular momentum into the reference frame.
   * The node vector is defined **relative to the reference plane**, not the orbital plane.

---

### 📐 Why this works

* The **longitude of the ascending node `Ω`** is defined as the angle from the reference direction (X-axis) **to `n_vec`**, measured **in the XY plane**.
* That’s why you need `K = [0, 0, 1]` — it allows you to find how **tilted the orbit is** relative to the base XY plane.

---

### 🖼️ Visual Analogy

Imagine you're looking at a tilted dinner plate (the orbital plane) from above (in the XY plane):

* The **plate’s normal** is `h_vec` (not vertical).
* The **intersection line between the plate and the table** is the **line of nodes**.
* That line lies on the table (XY plane), and you find it by doing `K × h_vec`.

---

### ✅ Summary

| Concept                    | Meaning                                                          |
| -------------------------- | ---------------------------------------------------------------- |
| `K = [0, 0, 1]`            | Unit vector normal to reference plane (Z-axis)                   |
| `h_vec`                    | Normal to orbital plane                                          |
| `n_vec = K × h_vec`        | Line of nodes, lies in XY plane                                  |
| `Ω = angle(n_vec, x-axis)` | Defines orientation of orbit's intersection with reference plane |

---

### 🔁 Final Intuition

* `K` is fixed → defines the **reference frame**.
* `h_vec` moves → defines the **orbital tilt**.
* `n_vec = K × h_vec` stays in the **reference plane**, marking the intersection line between orbit and the XY plane.

Would you like a diagram showing `h_vec`, `K`, and `n_vec` in 3D space?


In [33]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

def compute_orbital_elements_from_phase_space(star1, star2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # Average parallax to enforce consistent distance scale
    avg_parallax = (star1['parallax'] + star2['parallax']) / 2  # mas
    distance = (1000 / avg_parallax) * u.pc

    # SkyCoord objects
    c1 = SkyCoord(
        ra=star1['ra'] * u.deg,
        dec=star1['dec'] * u.deg,
        distance=distance,
        #distance = (1000 / star1['parallax']) * u.pc,
        pm_ra_cosdec=star1['pmra'] * u.mas/u.yr,
        pm_dec=star1['pmdec'] * u.mas/u.yr,
        radial_velocity=star1['radial_velocity'] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2['ra'] * u.deg,
        dec=star2['dec'] * u.deg,
        distance=distance,
        #distance = (1000 / star2['parallax']) * u.pc,
        pm_ra_cosdec=star2['pmra'] * u.mas/u.yr,
        pm_dec=star2['pmdec'] * u.mas/u.yr,
        radial_velocity=star2['radial_velocity'] * u.km/u.s
    )

    # Masses and CM quantities
    m1, m2 = star1['mass'], star2['mass']
    M = m1 + m2
    M_r = (m1 * m2) / M
    mu = G*M

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value

    print(f"Computed positions: {r1}, {r2}")
    print(f"Computed velocities: {v1}, {v2}")

    # Relative position and velocity in CM frame
    r_rel = r2 - r1
    v_rel = v2 - v1
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)


    # Angular momentum
    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)

    print(f"Computed anguar momentum: {h_vec}, abs: {h_mag:.3f}")

    # Specific mechanical energy
    E = 0.5*v_mag**2. - mu/r_mag
    E_total = E * M_r  # Total energy in the system
    print(f"Computed specific energy: {E:.3f}, Total energy: {E_total:.3f}")
    a = - mu/(2 * E)  # Semi-major axis

    print(f"Computed energy total: {E:.3f}")

    # Eccentricity vector
    e_vec = (np.cross(v_rel, h_vec) / mu) - (r_rel / r_mag) 
    e = np.linalg.norm(e_vec)

    # Inclination
    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    print(f"Computed node vector: {n_vec}, abs: {n_mag:.3f}")

    # Longitude of ascending node
    if n_mag != 0:
        Omega = np.degrees(np.arccos(n_vec[0] / n_mag))
        if n_vec[1] < 0:
            Omega = 360 - Omega
    else:
        Omega = 0

    # Argument of periapsis
    if n_mag != 0 and e > 1e-8:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly
    if e > 1e-8:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    # Orbital period (years)
    T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))


    print(f"Semi-major axis (pc): {a:.3f}, Eccentricity: {e:.3f}, Inclination (deg): {i_deg:.3f}")
    print(f"Longitude of ascending node (deg): {Omega:.3f}, Argument of periapsis (deg): {omega:.3f}, True anomaly (deg): {nu:.3f}")
    print(f"Orbital period (years): {T_yr:.3f}")
    print("")

    # Return the orbital elements as a dictionary

    return {
        'semi_major_axis_pc': a,
        'eccentricity': e,
        'inclination_deg': i_deg,
        'longitude_of_ascending_node_deg': Omega,
        'argument_of_periapsis_deg': omega,
        'true_anomaly_deg': nu,
        'orbital_period_yr': T_yr
    }




if not os.path.exists('results'):
    os.makedirs('results/')

result_orbital_elements_pair = {}
orbital_features = ['source_id', 'ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity', 'mass']

for i in range(len(binaries_members_1_MCM)):
    star1 = binaries_members_1_MCM.loc[i, orbital_features]
    star2 = binaries_members_2_MCM.loc[i, orbital_features]
        
    print(f"Computed orbital elements for the pair number {i}")

    # Compute the orbital elements from the phase space
    star1 = star1.to_dict()
    star2 = star2.to_dict()

    print(f"Star 1 mass: {star1['mass']}, Star 2 mass: {star2['mass']}")
    
    # Compute orbital elements
    orbital_elements = compute_orbital_elements_from_phase_space(star1, star2)
    
    result_orbital_elements_pair[i] = orbital_elements

# Convert the dictionary to a DataFrame
orbital_elements_df = pd.DataFrame.from_dict(result_orbital_elements_pair, orient='index')

if save:
    # Save the DataFrame to a CSV file
    orbital_elements_df.to_csv('results/My_results/orbital_elements_wRV.csv', index=False)

    # Save the results to CSV files
    #binaries_members_1_MCM.to_csv('results/My_results/binaries_members_1_MCM.csv', index=False)
    #binaries_members_2_MCM.to_csv('results/My_results/binaries_members_2_MCM.csv', index=False)


Computed orbital elements for the pair number 0
Star 1 mass: 5.001, Star 2 mass: 1.067
Computed positions: [ 65.40725813 102.63509329  53.80134436], [ 65.35299789 102.67485107  53.79142191]
Computed velocities: [ -2.28082755  21.09458294 -25.87487918], [ -4.9403736   17.52964374 -24.6893072 ]
Computed anguar momentum: [0.01176277 0.09071863 0.29917206], abs: 0.313
Computed specific energy: 10.210, Total energy: 8.978
Computed energy total: 10.210
Computed node vector: [-0.09071863  0.01176277  0.        ], abs: 0.091
Semi-major axis (pc): -0.001, Eccentricity: 54.164, Inclination (deg): 17.002
Longitude of ascending node (deg): 172.612, Argument of periapsis (deg): 331.776, True anomaly (deg): 358.286
Orbital period (years): nan

Computed orbital elements for the pair number 1
Star 1 mass: 0.6, Star 2 mass: 0.614
Computed positions: [67.25323604 97.92800663 54.67454611], [67.26778969 97.90263631 54.7020704 ]
Computed velocities: [  1.74965615  26.34274914 -20.46028289], [ -1.7416744   

C:\Users\bosca\AppData\Local\Temp\ipykernel_24156\1246114128.py:108: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_24156\1246114128.py:108: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_24156\1246114128.py:108: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_24156\1246114128.py:108: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_24156\1246114128.py:108: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\Users\bosca\AppData\Local\Temp\ipykernel_24156\1246114128.py:108: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))
C:\U